# Projekt 3 - Eksperymenty i analiza danych
## Autorzy
- Daniel Zdancewicz [145317]
- Alex Terentowicz [145419]

## Opis
t 3 polega na przeprowadzeniu eksperymentów z użyciem kilku metod i przeprowadzeniu
analizy uzyskanych wyników. Znajdź zbiór danych, który zawiera kilka kryteriów monotonicznych, ponad 100 alternatyw i co najmniej 2 klasy (jeśli problem ma więcej klas, możesz je zbinaryzować).

Wyobraźmy sobie, że jesteśmy instytucją zajmującą się utylizacją odpadów radioaktywnych. Naszym
zadaniem jest zaproponowanie modelu utylizacji takich odpadów dla 30-letniego programu nuklearnego.
Wybrano 3 miejsca na potencjalne składowisko, 3 scenariusze czasowe oraz 3 modele finansowania. Ze
względu na to, że ostatecznie klient może się nie zgodzić na optymalną dla nas strategię, dlatego chcielibyśmy uzyskać pełne rankingi wszystkich opcji. Zespół ekspertów ocenił wszystkie możliwe scenariusze przez nas rozważane (dane znajdują się w pliku [Nuclear waste management.csv](.


Krótko opisz zbiór danych, w tym opisy kryteriów. Dla tego zbioru danych wytrenuj następujące modele:
- Jeden prosty, interpretowalny model ML (np. XGBoost, rankSVM lub regresja logistyczna)
- Jeden interpretowalny model ANN-MCDA przedstawiony na zajęciach (ANN-Ch-Constr.
lub ANN-UTADIS)
- Złożony model sici neuronowej zawierającej kilka warstw ukrytych.


## Opis zbioru danych

### Opisy kryteriów


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train, test = (pd.read_csv(f"./data/loan_sanction_{p}.csv") for p in ["train", "test"])
train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [28]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Preprocessing

In [29]:
from sklearn.preprocessing import LabelEncoder

def preprocess(df):
    df = df.copy()
    df = df.dropna()
    encoded_cols = ["Gender", "Married", "Education", "Self_Employed", "Property_Area"]
    if "Loan_Status" in df.columns:
        encoded_cols.append("Loan_Status")
    df[encoded_cols] = df[encoded_cols].apply(LabelEncoder().fit_transform)
    return df

train, test = (preprocess(df) for df in [train, test])
train.describe()

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,0.820833,0.647917,0.202083,0.137500,5364.231250,1581.093583,144.735417,342.050000,0.854167,1.022917,0.691667
std,0.383892,0.478118,0.401973,0.344734,5668.251251,2617.692267,80.508164,65.212401,0.353307,0.776411,0.462287
min,0.000000,0.000000,0.000000,0.000000,150.000000,0.000000,9.000000,36.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,2898.750000,0.000000,100.000000,360.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,3859.000000,1084.500000,128.000000,360.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,0.000000,0.000000,5852.500000,2253.250000,170.000000,360.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,81000.000000,33837.000000,600.000000,480.000000,1.000000,2.000000,1.000000


In [30]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
5,LP001011,1,1,2,0,1,5417,4196.0,267.0,360.0,1.0,2,1


## Trening modeli

### Rank SVM — interpretowalny model ML

### ANN-Ch-Constr. — interpretowalny model ANN-MCDA

### Głęboka sieć neuronowa — model złożony.

## Eksperymenty
Wszystkie prezentowane wartości są do maksymalnie 4 miejsc po przecinku.

### Przedstawienie metryk

- Rank SVM
    - Accuracy
    - F1
    - AUC
- ANN-Ch-Constr.
    - Accuracy
    - F1
    - AUC
- Głęboka sieć neuronowa
    - Accuracy
    - F1
    - AUC


### Przedstawienie modelu
(np funkcje cząstkowe, schemat drzewa ...).


### Interpretacja
- Czy na podstawie uzyskanych parametrów możemy powiedzieć coś o preferencjach
użytkowników?
- Czy są jakieś kryteria, które nie mają żadnego wpływu, czy też mają wpływ decydujący.
- Czy istnieją jakieś progi preferencji?
- Czy istnieją oceny kryteriów, które są nierozróżnialne z punktu widzenia preferencji?


### Interpretacja techniką Partial Dependence Plot
Wykonaj interpretację modelu korzystając z conajmniej jednej techniki( Global Surrogate,
Partial Dependence Plot Permutation Feature Importance ...)


Dla 3 wybranych wariantów dokonaj wyjaśnienia decyzji.
- Znajdź minimalną zmianę w ocenach tak aby wariant został przypisany do innej klasy.
    – Spróbuj odpowiedzieć na to pytanie w sposób analityczny, opierając się tylko na
wartościach parametrów modelu i wyjaśnij, dlaczego taka zmiana jest minimalna
(bez próbkowania).
    - Wykonaj próbkowanie przestrzeni, nieznacznie zmieniając oceny, tak aby uzyskać
inną klasę. Czy wyniki zgadzają się z przewidywaniami teoretycznymi?
- Dokonaj wyjaśnienia predykcji za pomocą conajmniej jednej techniki (Anchors LIME,
SHAP, ... )


##### Możliwe narzędzia
Lista narzędzi, które zawierają różne techniki wyjaśniania predykcji i interpretacji modelu:
- Shapash
- Alibi
- Explainerdashboard
- DALEX
- eli5
- aix360